In [1]:
import sys
import os
from langchain_core.messages import HumanMessage, SystemMessage

# --- Setup Project Path ---
# This allows the notebook to find your 'src' directory and the 'hangman' package
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.hangman.providers.llmprovider import load_llm_provider, ModelOutput

# --- Test Configuration ---
CONFIG_PATH = "../config/config.yaml"
QWEN_PROVIDER_NAME = "qwen3_14b_local"
KIMI_PROVIDER_NAME = "kimi_k2_openrouter"

# --- Reusable Test Function ---
def test_provider(provider_name: str):
    """Loads a provider by name, invokes it with a test prompt, and prints the results."""
    print(f"Loading provider '{provider_name}'...")
    try:
        llm_provider = load_llm_provider(config_path=CONFIG_PATH, provider_name=provider_name)
        print("✅ LLM Provider loaded successfully.")
    except Exception as e:
        print(f"❌ Failed to load LLM Provider: {e}")
        return

    # Create a sample conversation to send to the model
    sample_messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content="What is the capital of France? Explain your reasoning process step-by-step.")
    ]
    
    print("\nInvoking model with a test prompt (thinking enabled)...")
    # Invoke the model, requesting the thinking trace
    output = llm_provider.invoke(sample_messages, thinking=True)

    # Print the structured output
    print("\n--- 🤔 Thinking Process ---")
    print(output.get("thinking") or "No thinking trace found (as expected for this model).")

    print("\n--- 💬 Final Response ---")
    print(output.get("response") or "No response found.")

In [2]:
test_provider(KIMI_PROVIDER_NAME)

Loading provider 'kimi_k2_openrouter'...
✅ LLM Provider loaded successfully.

Invoking model with a test prompt (thinking enabled)...


2025-09-03 10:14:53,457 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-03 10:15:29,429 - WARNING - No thinking tags found in the response. Treating as direct response. The response is:
--
Step 1: Identify the country in question  
The question asks about the capital of France.

Step 2: Recall or look up the official capital city of France  
From general geographic knowledge, the city designated as the capital of France is Paris.

Step 3: Verify that Paris is indeed the seat of the French national government  
Paris hosts the Élysée Palace (residence of the President), the National Assembly, the Senate, and most ministries, confirming its status as the political capital.

Step 4: Conclude  
Therefore, the capital of France is Paris.
--



--- 🤔 Thinking Process ---
No thinking trace found (as expected for this model).

--- 💬 Final Response ---
Step 1: Identify the country in question  
The question asks about the capital of France.

Step 2: Recall or look up the official capital city of France  
From general geographic knowledge, the city designated as the capital of France is Paris.

Step 3: Verify that Paris is indeed the seat of the French national government  
Paris hosts the Élysée Palace (residence of the President), the National Assembly, the Senate, and most ministries, confirming its status as the political capital.

Step 4: Conclude  
Therefore, the capital of France is Paris.


In [8]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage

load_dotenv()

api_key = os.environ.get("OPENROUTER_API_KEY")

model_name = "openai/gpt-oss-20b"
base_url = "https://openrouter.ai/api/v1"
api_key_env = "OPENROUTER_API_KEY"
temperature = 0.3
max_tokens = 16000

model_name = "openai/gpt-oss-20b"

client = ChatOpenAI(
    model=model_name,
    base_url=base_url,
    api_key=api_key,
    temperature=temperature,
    max_tokens=max_tokens,
)

response = client.invoke(
    [
        SystemMessage(content="You are a helpful assistant. Reasoning: High"),
        HumanMessage(content="What is the first prime number greater than 100000?")
    ],
)

print(response)

2025-09-03 10:40:40,376 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


content='The first prime number greater than\u202f100\u202f000 is **100\u202f003**.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5117, 'prompt_tokens': 97, 'total_tokens': 5214, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': None, 'id': 'gen-1756910440-8lSKmdvMmiBTtWvRtBSO', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--91fdbae4-8fe8-4b0a-b70b-0cd034dd0c05-0' usage_metadata={'input_tokens': 97, 'output_tokens': 5117, 'total_tokens': 5214, 'input_token_details': {}, 'output_token_details': {}}


In [9]:
response

AIMessage(content='The first prime number greater than\u202f100\u202f000 is **100\u202f003**.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5117, 'prompt_tokens': 97, 'total_tokens': 5214, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': None, 'id': 'gen-1756910440-8lSKmdvMmiBTtWvRtBSO', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--91fdbae4-8fe8-4b0a-b70b-0cd034dd0c05-0', usage_metadata={'input_tokens': 97, 'output_tokens': 5117, 'total_tokens': 5214, 'input_token_details': {}, 'output_token_details': {}})

In [10]:
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))  # uses OPENAI_API_KEY

stream = client.responses.stream(
    model="gpt-oss-20b",
    input=[{"role": "user", "content": "First prime > 100000?"}],
    reasoning={"effort": "high"},  # controls internal budget
    # (Optional) include other content in the response; by default you’ll get reasoning items
)

analysis_text = []
final_text = []

with stream as s:
    for event in s:
        # Raw CoT chunks:
        if event.type == "response.reasoning_text.delta":
            analysis_text.append(event.delta)
        if event.type == "response.reasoning_text.done":
            pass  # one turn of CoT is complete

        # Final assistant text:
        if event.type == "response.output_text.delta":
            final_text.append(event.delta)

print("ANALYSIS:\n", "".join(analysis_text))
print("\nFINAL:\n", "".join(final_text))

2025-09-03 11:17:26,219 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 400 Bad Request"


BadRequestError: Error code: 400 - {'error': {'message': "The requested model 'gpt-oss-20b' does not exist.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'model_not_found'}}

In [11]:
import os, requests, json

url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
  "Authorization": f"Bearer {os.environ['OPENROUTER_API_KEY']}",
  "Content-Type": "application/json"
}
payload = {
  "model": "openai/gpt-oss-20b",
  "messages": [{"role": "user", "content": "First prime > 100000?"}],
  "reasoning": { "effort": "high" }  # or {"enabled": true}
  # If you DON'T want CoT in the response: {"effort": "high", "exclude": True}
}

resp = requests.post(url, headers=headers, data=json.dumps(payload)).json()
msg = resp["choices"][0]["message"]
print("REASONING (raw):\n", msg.get("reasoning"))    # <-- CoT text when available
print("\nFINAL:\n", msg["content"])

REASONING (raw):
 We need to interpret the prompt: "First prime > 100000?" The user is asking for the first prime greater than 100,000. That means find smallest prime number that is strictly greater than 100,000. Let's calculate. Known primes around that range: Starting from 100,001 onwards. We can test primality. Let’s recall 100,003 is prime (I think). We must verify.

100,003: Check divisibility by small primes: 2 no, 3: sum digits 1+0+0+0+0+3=4 -> not divisible by 3. 5 no. 7: 100,003 mod 7? 7*14286=100002, remainder 1. So not divisible. 11: 11*9091=100,001, remainder 2. 13: 13*7692=99996, remainder 7. 17: 17*5882=100, - 17*5882=100 - Actually 5882*17=100, - let's compute: 5882*10=58820, plus 5882*7=41174, sum 99994, remainder 9. 19: 19*5263=99997, remainder 6. 23: 23*4348=100, - 23*4348: 4348*20=86960, plus 4348*3=13044 => total 100,004. So 100,003 would be 100004 -1, remainder -1 -> not. 29: 29*3448=100, - 29*3448 = 100, - 3448*30=103440 less 3448=100, - 103440 - 3448 = 100 -? Let

In [29]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.environ.get("OPENROUTER_API_KEY"),
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="openai/gpt-oss-20b",
  messages=[
    {
      "role": "user",
      "content": "Which is the first prime number greater than 100000?"
    }
  ]
)
print(completion.choices[0].message.content)

2025-09-03 11:31:17,183 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


The smallest prime number that is larger than 100 000 is  

**100 003**.


In [30]:
raw = completion.model_dump()  # or: json.loads(resp.model_dump_json())
msg = raw["choices"][0]["message"]

print("REASONING:\n", msg.get("reasoning"))  # raw analysis CoT if the provider surfaces it
print("\nFINAL:\n", msg.get("content"))

REASONING:
 We are asked: Which is the first prime number greater than 100000? The answer should be the smallest prime > 100000. Let's find primes > 100,000. 100,003? 100,003 is known to be prime? Let's check small divisibility: 100003 mod 3 = 1, mod 5 = 3, mod 7? 7*14286=100002, so 100003 mod 7 = 1. mod 11? 11*9091=999... 11*9091=999... Wait 9091*11 = 999... 9091*11 = 999... Let's check: 9091*11 = 999... Actually 9091 * 10 = 90910, plus 9091 = 999... 90910 + 9091 = 999... 90910+9091=999... 90910+9000=99910, plus 91 = 100001. So 11*9091 = 999... Wait let's compute precisely: 9091*11 = 9091*10 + 9091 = 90910 + 9091 = 999... 90910+9000=99910, plus 91=100001. So 11*9091 = 999... 90910+9091=999... Actually 90910+9091 = 999... 90910+9000=99910, +91=100001. So 11*9091 = 999... Wait 90910 + 9091 = 999... yes equals 999... Let's sum again: 90910 + 9091 = 999... 90910 + 9000 = 99910, +91=100001. So 11*9091 = 100001. So 100001 is composite: 11 * 9091. 100003 would be 100001+2. Need to check divi

In [ ]:
completion.

{'id': 'gen-1756913476-UK0s36fPjEAeFhI9AczG',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The smallest prime number that is larger than\u202f100\u202f000 is  \n\n**100\u202f003**.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning="We are asked: Which is the first prime number greater than 100000? The answer should be the smallest prime > 100000. Let's find primes > 100,000. 100,003? 100,003 is known to be prime? Let's check small divisibility: 100003 mod 3 = 1, mod 5 = 3, mod 7? 7*14286=100002, so 100003 mod 7 = 1. mod 11? 11*9091=999... 11*9091=999... Wait 9091*11 = 999... 9091*11 = 999... Let's check: 9091*11 = 999... Actually 9091 * 10 = 90910, plus 9091 = 999... 90910 + 9091 = 999... 90910+9091=999... 90910+9000=99910, plus 91 = 100001. So 11*9091 = 999... Wait let's compute precisely: 9091*11 = 9091*10 + 9091 = 90910 + 9091 = 999... 90910+9000=99910, plus 91=10

In [13]:
completion

ChatCompletion(id='gen-1756912829-vYs4SBLDRPxifP7aNQoE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The question “What is the meaning of life?” is one of the oldest and most profound inquiries, and there are many ways to approach it. Here are a few broad perspectives that people often find useful:\n\n| **Perspective** | **Key Ideas** | **What It Might Mean for You** |\n|-----------------|---------------|--------------|\n| **Philosophical** | - **Existentialism**: Life has no pre‑written meaning; we create it through choices.<br>- **Essentialism**: Some see a core essence (e.g., virtues, fulfillment) that gives purpose.<br>- **Stoicism**: Find meaning by aligning with nature, cultivating inner peace. | You might focus on living authentically, making deliberate choices, or cultivating resilience and gratitude. |\n| **Religious / Spiritual** | - **Monotheistic faiths**: Life is a test or a journey toward communion with God.<br>- **

In [26]:
completion.__dict__

{'id': 'gen-1756913238-4mcCqEE3C4ncILtVMkFb',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The smallest prime number that is greater than 100\u202f000 is **100\u202f003**.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning="I need to find smallest prime > 100000. Let's think. 100000 is even, next odd numbers: 100001 to 100003 etc. Let's check.\n\nWe can quickly approximate: Check 100001. 100001 = maybe divisible by 11? 100001 / 11 = 9091? Wait 11*9091=100001. Yes 100001 = 11 * 9091, so composite.\n\n100003: check small primes. 100003 mod 3: sum digits=4, not multiple of 3. mod 5 no. mod 7? 7*14286=100002, remainder 1. Not divisible. mod 11? 11*9091=100001, remainder 2. mod 13? 13*7692=99996, remainder 7. mod 17? 17*5882=99994, remainder 9. mod 19? 19*5263=100, 19*5263= 19*5000=95000, 19*263=4997, sum=99997, remainder 6. mod 23? 23*4348=100004, overshoot. 23*4347=23*43

In [ ]:
from openai import OpenAI
import os

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
)

resp = client.chat.completions.create(
    model="openai/gpt-oss-20b",
    messages=[{"role": "user", "content": "First prime > 100000?"}],
)

raw = resp.model_dump()  # or: json.loads(resp.model_dump_json())
msg = raw["choices"][0]["message"]

print("REASONING:\n", msg.get("reasoning"))  # raw analysis CoT if the provider surfaces it
print("\nFINAL:\n", msg.get("content"))

2025-09-03 11:30:34,446 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


REASONING:
 We need to answer: "First prime > 100000?" The user likely expects the prime number that is the next after 100000. Let's find: 100001? 100001 is 11*9091. 100003? 100003 is prime? Let's check. We can recall known primes: 100000 is composite. The next prime >100000 is 100003. Check divisibility: sqrt(100003) ~316. So test primes up to 317. Use quick mental check: 100003 mod 3 = 1 (sum digits 1+? Actually 1+0+0+0+0+3 =4, not divisible). mod5 not 0. mod7: 100003 /7 = 14286.14? 7*14286=999... 7*14286=999... Wait 14286*7 = 999... 14000*7=98000, 286*7=2002, sum=100002, so remainder 1. not divisible. mod11: 11*9091=100001 => 100003-100001=2 so not divisible. mod13: 13*7692=99996, remainder 7. mod17: 17*5882=100, 17*5882=99994, remainder 9. mod19: 19*5264=100, 19*5264=999... 19*5264=100016? Wait 5264*19=5264*20-5264=105280-5264=100016. remainder -? 100003-100016=-13 no. So not divisible. mod23: 23*4348=100 similarly? 4348*23=4348*20+4348*3=86960+13044=100004, remainder -1 maybe. Not

In [36]:
import os
os.environ["LANGCHAIN_OPENAI_USE_RESPONSES_API"] = "false"  # must be BEFORE imports

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(
    model="openai/gpt-oss-20b",
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    temperature=0.2,
    max_tokens=2000,
    # do NOT pass use_legacy_completions here (your LC version forwards it incorrectly)
    model_kwargs={"reasoning": {"effort": "high", "include": True}},
)

resp = llm.invoke([HumanMessage(content="First prime > 100000?")])

print("FINAL:", resp.content)
print("REASONING:", resp.additional_kwargs.get("reasoning"))

/home/mila/b/baldelld/scratch/hangman/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3639: UserWarning: Parameters {'reasoning'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):
2025-09-03 11:37:51,746 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/responses "HTTP/1.1 200 OK"


AttributeError: 'str' object has no attribute 'error'

In [35]:
llm.__dict__

{'name': None,
 'cache': None,
 'verbose': False,
 'callbacks': None,
 'tags': None,
 'metadata': None,
 'custom_get_token_ids': None,
 'callback_manager': None,
 'rate_limiter': None,
 'disable_streaming': False,
 'client': <openai.resources.chat.completions.completions.Completions at 0x7fe3c55e2b90>,
 'async_client': <openai.resources.chat.completions.completions.AsyncCompletions at 0x7fe3c4e5f010>,
 'root_client': <openai.OpenAI at 0x7fe3c55b8410>,
 'root_async_client': <openai.AsyncOpenAI at 0x7fe3c4e5ff90>,
 'model_name': 'openai/gpt-oss-20b',
 'temperature': 0.2,
 'model_kwargs': {'use_legacy_completions': True},
 'openai_api_key': SecretStr('**********'),
 'openai_api_base': 'https://openrouter.ai/api/v1',
 'openai_organization': None,
 'openai_proxy': None,
 'request_timeout': None,
 'stream_usage': False,
 'max_retries': None,
 'presence_penalty': None,
 'frequency_penalty': None,
 'seed': None,
 'logprobs': None,
 'top_logprobs': None,
 'logit_bias': None,
 'streaming': False